Meeting notes:

##### Make diag.bib update code (notebook)
- Compare titles of bib and semantic scholar
- Match
- If match:
    - Add semantic scholar ID to diag bib
    - Update gscites field from bib entry
- If no match:
    - Check best matches, if DOI and Arxiv --> chose DOI 
    - Define right fields (look at code James) + citations + semantic scolar ID 
    - Add 'automatically added' field so we can trace back automation errors more easilty
    - Add to bib   
    
    
##### Make script that can be used by DIAG members
- Add doi/arxiv
- if DOI check if Arxiv exists, if so --> remove Arxiv
- Sort right fields from DOI --> add fields
- Search for semantic scholar ID
- Print fields so user can check it
- Add to bib
- Manual commit push

##### Make script that checks all semantic scholar IDs and updates gscites
- github action overnight?

# Libraries

In [23]:
import os
import numpy as np
import pandas as pd

import requests
# from bibreader import parse_bibtex_file, get_bib_blocks
from bib_handling_code.processbib import read_bibfile
from bib_handling_code.processbib import save_to_file
from difflib import SequenceMatcher


# Example from url giving 100 papers from one of Bram's semantic scolar IDs

In [22]:
staff_id = 8038506
staff_ids = [8038506]

In [21]:
url=f'https://api.semanticscholar.org/graph/v1/author/{staff_id}/papers?fields=year,title,authors,externalIds,citationCount&limit=100'

r = requests.get(url)
ss_staff_data = r.json()['data']

In [58]:
# ss_staff_data[0]

# Function to get data from DOI/ArXiv

In [5]:
def get_doi_csl(doi):
    response = requests.get(
        f"https://doi.org/{doi}",
        headers={"Accept": "application/vnd.citationstyles.csl+json"},
        timeout=5,
    )
    response.raise_for_status()
    return response.json()

# Example

In [6]:
'arXiv' in '10.48550/arXiv.2302.01738' 

True

In [7]:
ss_staff_data[2]['externalIds']

{'ArXiv': '2302.01738',
 'DBLP': 'journals/corr/abs-2302-01738',
 'DOI': '10.48550/arXiv.2302.01738',
 'CorpusId': 256598301}

In [8]:
found_arxiv_doi = False
found_regular_doi = False

In [9]:
#ArXiv
for ss_staff_entry in ss_staff_data:
    if 'DOI' in ss_staff_entry['externalIds']:
        if 'arXiv' in ss_staff_entry['externalIds']['DOI']:
            arxiv_doi = ss_staff_entry['externalIds']['DOI']
            found_arxiv_doi = True
    if found_arxiv_doi:
        break
#DOI
for ss_staff_entry in ss_staff_data:
    if 'DOI' in ss_staff_entry['externalIds']:
        if 'arXiv' not in ss_staff_entry['externalIds']['DOI']:
            regular_doi = ss_staff_entry['externalIds']['DOI']
            found_regular_doi = True
    if found_regular_doi:
        break

print(arxiv_doi)
print(regular_doi)

10.48550/arXiv.2302.01790
10.1007/978-3-031-28241-6_18


In [10]:
doi = regular_doi

In [11]:
doi_data = get_doi_csl(doi)
doi_data

{'indexed': {'date-parts': [[2023, 3, 16]],
  'date-time': '2023-03-16T04:50:23Z',
  'timestamp': 1678942223397},
 'publisher-location': 'Cham',
 'reference-count': 10,
 'publisher': 'Springer Nature Switzerland',
 'isbn-type': [{'value': '9783031282409', 'type': 'print'},
  {'value': '9783031282416', 'type': 'electronic'}],
 'license': [{'start': {'date-parts': [[2023, 1, 1]],
    'date-time': '2023-01-01T00:00:00Z',
    'timestamp': 1672531200000},
   'content-version': 'tdm',
   'delay-in-days': 0,
   'URL': 'https://www.springernature.com/gp/researchers/text-and-data-mining'},
  {'start': {'date-parts': [[2023, 1, 1]],
    'date-time': '2023-01-01T00:00:00Z',
    'timestamp': 1672531200000},
   'content-version': 'vor',
   'delay-in-days': 0,
   'URL': 'https://www.springernature.com/gp/researchers/text-and-data-mining'}],
 'content-domain': {'domain': ['link.springer.com'],
  'crossmark-restriction': False},
 'published-print': {'date-parts': [[2023]]},
 'DOI': '10.1007/978-3-031-

In [12]:
[print(key) for key in doi_data.keys()];

indexed
publisher-location
reference-count
publisher
isbn-type
license
content-domain
published-print
DOI
type
created
page
update-policy
source
is-referenced-by-count
title
prefix
author
member
published-online
reference
container-title
original-title
link
deposited
score
resource
subtitle
short-title
issued
ISBN
references-count
URL
relation
ISSN
published
assertion


# Import diag.bib

In [13]:
path_diag_bib = os.path.join('..', 'diag.bib')
diag_bib_raw = read_bibfile(None, path_diag_bib) # I changed the code in such a way that IF I give a second argument, it uses the second argument as a full path

In [14]:
len(diag_bib_raw)

2783

# Gather ss and bib data

## Semantic scolar

In [48]:
ss_data = []
# get data for staff member
for staff_id in staff_ids:
    # get semantic scolar data
    url=f'https://api.semanticscholar.org/graph/v1/author/{staff_member_id}/papers?fields=year,title,authors,externalIds,citationCount&limit=100'
    r = requests.get(url)
    ss_staff_data = r.json()['data']
    
    # loop over semantic scolar data entries
    for ss_staff_entry in ss_staff_data:
        ss_id = None
        ss_title = None
        ss_doi = None
        ss_citations = None

        if 'paperId' in ss_staff_entry:
            ss_id = ss_staff_entry['paperId']
        if 'title' in ss_staff_entry:
            ss_title = ss_staff_entry['title']
        if 'DOI' in ss_staff_entry['externalIds']:
            ss_doi = ss_staff_entry['externalIds']['DOI']
        if 'citationCount' in ss_staff_entry:
            ss_citations = ss_staff_entry['citationCount']

        ss_data.append([ss_id, ss_title, ss_doi, staff_id, ss_citations])
ss_columns = ['ss_id', 'title', 'doi', 'staff_id', 'ss_citations']
df_ss_data = pd.DataFrame(ss_data, columns=ss_columns)

In [49]:
df_ss_data.head()

,ss_id,title,doi,staff_id,ss_citations
0,1898998f27ce0750a42f8f3ca2ed7e292f05aee2,Understanding metric-related pitfalls in image...,10.48550/arXiv.2302.01790,8038506,3
1,362c510dec0d566d22d5be3af0519fc7eec8bb86,Uncertainty-Aware Multiple-Instance Learning f...,None,8038506,0
2,7981606bf8110ec6cc64baa22d694096f7862939,AIROGS: Artificial Intelligence for RObust Gla...,10.48550/arXiv.2302.01738,8038506,2
3,e738d94746b0f33b1c48737022ab186618d4d4cd,SimpleRad: Patient-Friendly Dutch Radiology Re...,10.1007/978-3-031-28241-6_18,8038506,0
4,2d49fb82fd715a213e2149f9f08dd2fb4749b235,Challenges in digital medicine applications in...,10.1038/s41467-022-30728-3,8038506,3


## DIAG bib

In [56]:
bib_data = []
for bib_entry in diag_bib_raw:
    bibkey = bib_entry.key
    
    bib_title = None
    bib_doi = None
    bib_gscites = None
    
    if 'title' in bib_entry.fields:
        bib_title = bib_entry.fields['title'].strip('{ }')
    if 'doi' in bib_entry.fields:
        bib_doi = bib_entry.fields['doi'].strip('{ }')
    if 'gscites' in bib_entry.fields:
        bib_gscites = bib_entry.fields['gscites'].strip('{ }')
    
    bib_data.append([bibkey, bib_title, bib_doi, bib_gscites])
bib_columns = ['bibkey', 'title', 'doi', 'gs_citations']
df_bib_data = pd.DataFrame(bib_data, columns=bib_columns)

In [59]:
df_bib_data.to_csv('script_data/temp_diag_bib_summary.csv', index=False)

In [57]:
df_bib_data.tail()

,bibkey,title,doi,gs_citations
2778,Sadr23,Deep Learning for Detection of Periapical Radi...,10.1016/j.joen.2022.12.007,None
2779,Anto23,Retrospective validation of nodule management ...,None,None
2780,Sido23,Multi-source data approach for personalized ou...,10.1007/s10654-023-00975-9,None
2781,Graa23,MRI image features with an evident relation to...,10.1007/s00586-023-07602-x,None
2782,Graa22,Segmentation of vertebrae and intervertebral d...,10.1117/12.2611423,None


# Loop for matching titles

In [ ]:
for title in df_ss_data['']

In [ ]:
bib_title = bib_entry.fields['title'].strip('{ }')
ss_title = ss_entry

In [ ]:
all_types = []
for entry in diag_bib_raw:
    all_types.append(entry.type)
np.unique(all_types, return_counts=True)

In [ ]:
selected_entries = []
for entry in diag_bib_raw:
    if entry.type in ['article']: #['article', 'book', 'conference', 'inbook', 'inproceedings']
        selected_entries.append(entry)
        print(entry.fields)

In [ ]:
[print(key) for key in selected_entries[0].fields.keys()];